In [ ]:
pip install firecrawl-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.7/168.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
# pip install firecrawl-py

from firecrawl import Firecrawl

firecrawl = Firecrawl(api_key="fc-ab42537a97394fa3854f1236588d59a6")

In [ ]:
doc = firecrawl.scrape("https://chandru22.vercel.app/", formats=["markdown", "html"])
print(doc)

markdown="![Profile](https://chandru22.vercel.app/profile.jpg?height=320&width=320)\n\n# Chandru\n\nAI enthusiast and Full Stack Developer\n\nI build exceptional and accessible digital experiences for the web.\n\n[GitHub](https://github.com/Chandrut22)[LinkedIn](https://www.linkedin.com/in/chandrut22/)[LeetCode](https://leetcode.com/u/Chandrutd2004/)[GeeksForGeeks](https://www.geeksforgeeks.org/user/chandru22/)[Resume](https://chandru22.vercel.app/Chandru_Resume.pdf) [Get In Touch](https://chandru22.vercel.app/#contact)\n\n## Skills & Expertise\n\nI've spent several years honing my skills across various technologies. Here's an overview of my technical expertise and proficiency levels.\n\nFrontend Development\n\nBuilding responsive and accessible user interfaces\n\nC95%\n\nJava90%\n\nPython85%\n\nHTML/CSS75%\n\nJavaScript70%\n\nBackend Development\n\nCreating robust and scalable server-side applications\n\nDjango85%\n\nSpringBoot80%\n\nNodeJs75%\n\nMySql85%\n\nMongoDB65%\n\nTools & Tech

In [ ]:
print(doc.markdown)

![Profile](https://chandru22.vercel.app/profile.jpg?height=320&width=320)

# Chandru

AI enthusiast and Full Stack Developer

I build exceptional and accessible digital experiences for the web.

[GitHub](https://github.com/Chandrut22)[LinkedIn](https://www.linkedin.com/in/chandrut22/)[LeetCode](https://leetcode.com/u/Chandrutd2004/)[GeeksForGeeks](https://www.geeksforgeeks.org/user/chandru22/)[Resume](https://chandru22.vercel.app/Chandru_Resume.pdf) [Get In Touch](https://chandru22.vercel.app/#contact)

## Skills & Expertise

I've spent several years honing my skills across various technologies. Here's an overview of my technical expertise and proficiency levels.

Frontend Development

Building responsive and accessible user interfaces

C95%

Java90%

Python85%

HTML/CSS75%

JavaScript70%

Backend Development

Creating robust and scalable server-side applications

Django85%

SpringBoot80%

NodeJs75%

MySql85%

MongoDB65%

Tools & Technologies

Supporting skills and development tools

G

In [ ]:
print(doc.html)

<!DOCTYPE html><html lang="en"><body class="__variable_f910ec __variable_c8ca7d __variable_6d5c28 font-sans text-accent-black bg-background-base overflow-x-clip" style=""><main class="overflow-x-clip"><div class="container p-12 relative z-[102]"><div class="p-10 rounded-10 overflow-hidden relative text-body-medium text-center bg-black-alpha-4 text-accent-black">We just raised our Series A and shipped Firecrawl /v2 🎉.<!-- --> <a class="text-label-medium" href="https://www.firecrawl.dev/blog/firecrawl-v2-series-a-announcement">Read the blog.</a></div><div class="bottom-0 absolute h-1 w-screen left-[calc(50%-50vw)] bg-border-faint"></div><svg fill="none" height="21" viewBox="0 0 22 21" width="22" xmlns="http://www.w3.org/2000/svg" class="pointer-events-none contain-[layout,paint] absolute absolute -bottom-10 -left-[10.5px]"><path d="M10.5 4C10.5 7.31371 7.81371 10 4.5 10H0.5V11H4.5C7.81371 11 10.5 13.6863 10.5 17V21H11.5V17C11.5 13.6863 14.1863 11 17.5 11H21.5V10H17.5C14.1863 10 11.5 7.31

In [ ]:
import os
from typing import Literal
from IPython.display import Image, display

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END

# -- DuckDuckGo import: try a resilient import (API exposed via langchain_community) --
try:
    # Common convenience import
    from langchain_community.tools import DuckDuckGoSearchResults
except Exception:
    # Fallback to the direct module path if package layout differs
    from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults

# (DuckDuckGoSearchResults docs / API reference). :contentReference[oaicite:1]{index=1}

# ----------------------------
# Config / LLM
# ----------------------------
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"  # set your key
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")  # a free/light-weight Gemini model

# ----------------------------
# Tools
# ----------------------------

# 1) local SEO optimizer tool (simple deterministic function for demonstration)
def seo_optimizer_tool_impl(args: dict) -> str:
    """
    Input: {"query": "<user request or URL or topic>"}
    Returns: a short structured string containing the optimized query and justification.
    """
    raw = args.get("query", "").strip()
    # Very simple heuristic example — in production you'd call a more sophisticated function
    optimized = f"{raw} latest research {2025} actionable keywords"
    justification = (
        "Added 'latest research 2025' for freshness; included 'actionable keywords' to "
        "encourage result pages that provide practical, implementable SEO tips."
    )
    # Return a clearly-parsable block (LLM will receive this as a tool observation)
    return f"Optimized Query: {optimized}\nJustification: {justification}"

# 2) DuckDuckGo web search tool (langchain_community)
ddg_search = DuckDuckGoSearchResults(max_results=5)

# Map by name so the tool_node can execute dynamically
tools_by_name = {
    "seo_optimizer": seo_optimizer_tool_impl,
    "web_search": ddg_search,  # runnable/BaseTool-like object
}

# Tool descriptors for LLM's function-calling awareness (the LLM should be told how to call them)
tool_descriptors = [
    {
        "name": "seo_optimizer",
        "description": "Optimize a user request (site URL or SEO question) into a search-friendly query and a short justification.",
        "args_schema": {"query": {"type": "string", "description": "User request or URL"}}
    },
    {
        "name": "web_search",
        "description": "Run a DuckDuckGo web search for a given query and return top results.",
        "args_schema": {"query": {"type": "string", "description": "Search query"}}
    },
]

# Bind tools to the LLM so the model knows their schemas (API may be beta/stable per provider)
llm_with_tools = llm.bind_tools(tool_descriptors)

# ----------------------------
# Nodes
# ----------------------------
def llm_call(state: MessagesState):
    """
    The LLM runs and may call tools. We seed the LLM with the 'SEO Optimization Agent'
    instructions (system prompt uses the user's role/goal/instructions).
    """
    system_prompt = SystemMessage(
        content=(
            "role=\"SEO Optimization Assistant\",\n"
            "goal=\"Optimize websites for better search engine ranking and increased organic traffic.\",\n"
            "instructions=\"\"\"\n"
            "You are an SEO Optimization Agent.\n\n"
            "Given a website URL or specific SEO-related question, provide a comprehensive SEO analysis and improvement suggestions.\n"
            "Steps (must be followed in this order):\n"
            "1) Call the 'seo_optimizer' tool with the user's raw request to produce an optimized search query + justification.\n"
            "2) Call the 'web_search' tool with the optimized query to gather recent references and examples.\n"
            "3) Produce a final SEO plan in Markdown covering: Keyword Research, On-Page, Off-Page, Technical SEO, Content Strategy.\n\n"
            "Use the DuckDuckGo tool for research to improve suggestions when needed. Ask for more details if required.\n"
            "Provide step-by-step recommendations in markdown-friendly format.\n"
            "\"\"\""
        )
    )

    # combine system prompt + conversation messages from state
    prompt_messages = [system_prompt] + state["messages"]
    return {"messages": [llm_with_tools.invoke(prompt_messages)]}


def tool_node(state: dict):
    """
    Executes tools requested by the LLM's last message.
    Supports both local functions and runnable tools (DuckDuckGoSearchResults).
    """
    result = []
    last_msg = state["messages"][-1]
    for tool_call in getattr(last_msg, "tool_calls", []):
        name = tool_call["name"]
        args = tool_call.get("args", {})
        tool = tools_by_name.get(name)
        # local python tool (callable)
        if callable(tool) and not hasattr(tool, "invoke") and not hasattr(tool, "run"):
            observation = tool(args)
        else:
            # For runnable tools (like DuckDuckGoSearchResults), pass the query string.
            query = args.get("query") if isinstance(args, dict) else args
            # try common call patterns
            if hasattr(tool, "invoke"):
                observation = tool.invoke(query)
            elif hasattr(tool, "run"):
                observation = tool.run(query)
            else:
                # fallback: attempt to call directly
                observation = tool(query)
        # Wrap the observation as a ToolMessage so the LLM can read it in the next step
        result.append(ToolMessage(content=str(observation), tool_call_id=tool_call.get("id")))
    return {"messages": result}


# ----------------------------
# Conditional edge
# ----------------------------
def should_continue(state: MessagesState) -> Literal["Action", END]:
    """
    If the LLM asked to call a tool, route to 'environment' node (Action),
    otherwise finish the graph and return the final answer.
    """
    last_message = state["messages"][-1]
    if getattr(last_message, "tool_calls", None):
        return "Action"
    return END


# ----------------------------
# Build & compile graph
# ----------------------------
agent_builder = StateGraph(MessagesState)
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("environment", tool_node)

agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {"Action": "environment", END: END}
)
agent_builder.add_edge("environment", "llm_call")

agent = agent_builder.compile()

# Visualize (optional)
try:
    display(Image(agent.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    pass

# ----------------------------
# Example run
# ----------------------------
sample = HumanMessage(content="Analyze https://example.com and suggest SEO improvements focusing on technical SEO and content.")
response_state = agent.invoke({"messages": [sample]})

# Print each message returned in the final state (tool outputs + LLM final message)
for m in response_state["messages"]:
    try:
        m.pretty_print()
    except Exception:
        print(str(m))
